
# Organisation of code submissions and standardisation to a common interface

## General structure
Code submissions are located in the src/original folder, where submissions are named as `<initials>_<institution>`. Due to code submissions having different authors, it is expected that they all vary in their usage, inputs, and outputs. In order to facilitate testing in a larger scale, a common interface has been created in the form of the `OsipiBase` class (src/wrappers). This class acts as a parent class for standardised versions of the different code submissions. Together, they create the common interface of function calls and function outputs that allows us to perform mass testing, but also creates easy usage.

The src/standardized folder contains the standardised version of each code submission. Here, a class is created following a naming convention (`<initials>_<institution>_<algorithm name>`), with `__init__()` and `ivim_fit()` methods that integrate well with the OsipiBase class. The idea is that every submitted fitting algorithm should be initialised in the same way, and executed in the same way.




## The standardized versions
The standardised versions of each submission is a class that contains two methods. These classes inherit the functionalities of `OsipiBase`.

### `__init__()`
The `__init__()` method ensures that the algorithm is initiated correctly in accordance with OsipiBase. Custom code is to be inserted below the `super()` call. This method should contain any of the neccessary steps for the following `ivim_fit()` method to only require signals and b-values as input.

Below is an example from src/standardized/IAR_LU_biexp.py

In [ ]:
def __init__(self, bvalues=None, thresholds=None, bounds=None, initial_guess=None, weighting=None, stats=False):
    """
        Everything this algorithm requires should be implemented here.
        Number of segmentation thresholds, bounds, etc.
            
        Our OsipiBase object could contain functions that compare the inputs with
        the requirements.
    """
    super(IAR_LU_biexp, self).__init__(bvalues, thresholds, bounds, initial_guess) ######## On this line, change "IAR_LU_biexp" to the name of the class

    ######## Your code below #########
        
    # Check the inputs
        
    # Initialize the algorithm
    if self.bvalues is not None:
        bvec = np.zeros((self.bvalues.size, 3))
        bvec[:,2] = 1
        gtab = gradient_table(self.bvalues, bvec, b0_threshold=0)
            
        self.IAR_algorithm = IvimModelBiExp(gtab)
    else:
        self.IAR_algorithm = None


### `ivim_fit()`
The purpose of this method is to take a singe voxel signal and b-values as input, and return IVIM parameters as output. This is where most of the custom code will go that is related to each individual code submission. The idea here is to have calls to submitted functions in the src/originals folder. This ensures that the original code is not tampered with. However if required, the original code could be just pasted in here as well.

Below is an example from src/standardized/IAR_LU_biexp.py

In [ ]:

def ivim_fit(self, signals, bvalues, **kwargs):
    """Perform the IVIM fit

    Args:
        signals (array-like)
        bvalues (array-like, optional): b-values for the signals. If None, self.bvalues will be used. Default is None.

    Returns:
        _type_: _description_
    """
        
    if self.IAR_algorithm is None:
        if bvalues is None:
            bvalues = self.bvalues
        else:
            bvalues = np.asarray(bvalues)
            
        bvec = np.zeros((bvalues.size, 3))
        bvec[:,2] = 1
        gtab = gradient_table(bvalues, bvec, b0_threshold=0)
            
        self.IAR_algorithm = IvimModelBiExp(gtab)
            
    fit_results = self.IAR_algorithm.fit(signals)
        
    results = {}
    results["f"] = fit_results.model_params[1]
    results["D*"] = fit_results.model_params[2]
    results["D"] = fit_results.model_params[3]
        
    return results


## The `OsipiBase` class
The usage of the OsipiBase class mainly consists of running the osipi_fit() method. In this method, the inputs from `__init__()` of the standardised version of a code submission, and the signals and b-values input to `osipi_fit()` is processed and fed into the `ivim_fit()` function.

It is the `osipi_fit()` method that provides the common interface for model fitting. As one may note, `ivim_fit()` takes a single voxel as input. `OsipiBase.osipi_fit()` supports multidimensional inputs, which is then iteratively fed into `ivim_fit()`, and returns a corresponding output. Support for future types of input will be implemented here. This ensures that the `ivim_fit()` method can be written as simply as possible, which simplifies the inclusion of new code submissions into the standard interface.



## Example usage of standardized version of an algorithm
### Using the standardized version directly
The standardised versions can be used directly by
1. Importing the class
2. Initialising the object with the required parameters, e.g. `IAR_LU_biexp(bounds=[(0, 1), (0.005, 0.1), (0, 0.004)])`
3. Call `osipi_fit(signals, bvalues)` for model fitting

### Using the `OsipiBase` class with algorithm names
Standardised versions can also be initiated using the OsipiBase.osipi_initiate_algorithm() method.

1. Import `OsipiBase`
2. Initiate `OsipiBase` with the algorithm keyword set to the standardised name of the desired algorithm e.g., `OsipiBase(algorithm=IAR_LU_biexp)`
3. Call `osipi_fit()` for model fitting